# **Imports**

In [ ]:
!pip install numpy gensim

In [ ]:
!pip install gensim

In [ ]:
!wget https://archive.org/download/full_grams_cbow_300_twitter/full_grams_cbow_300_twitter.zip
!unzip full_grams_cbow_300_twitter.zip

In [ ]:
import joblib
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
pip install pyarabic

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/ArSarcasm_train.csv")
df.head()

,dialect,sarcasm,sentiment,original_sentiment,tweet,source
0,gulf,False,negative,negative,"""نصيحه ما عمرك اتنزل لعبة سوبر ماريو مش زي ما ...",semeval
1,msa,False,neutral,positive,"""#نادين_نسيب_نجيم ❤️❤️❤️مجلة #ماري_كلير 💭#ملكة...",semeval
2,egypt,False,neutral,neutral,"""@Alito_NBA اتوقع انه بيستمر""",semeval
3,levant,True,neutral,negative,"""@KSA24 يعني ""بموافقتنا"" لأن دمشق صايرة موسكو""",semeval
4,msa,False,neutral,negative,"""RT @alaahmad20: قائد في الحرس يعترف بفقدان ال...",semeval


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8437 entries, 0 to 8436
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   dialect             8437 non-null   object
 1   sarcasm             8437 non-null   bool  
 2   sentiment           8437 non-null   object
 3   original_sentiment  8437 non-null   object
 4   tweet               8437 non-null   object
 5   source              8437 non-null   object
dtypes: bool(1), object(5)
memory usage: 337.9+ KB


In [ ]:
import re
from pyarabic.araby import strip_tashkeel, strip_tatweel

# **Pipeline**

In [ ]:
import re

def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
import nltk
nltk.download('punkt')

def tokenize_tweet(tweet):
 
    tokens = nltk.word_tokenize(tweet)
  
    return tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def remove_punctuation(tokens):
    # Remove punctuation from the tokens using a regular expression
    no_punct_tokens = [re.sub(r'[^a-zA-Z؀-ۿ]', '', token) for token in tokens]
    
    # Remove any empty tokens using a loop
    new_tokens = []
    for token in no_punct_tokens:
        if token:
            new_tokens.append(token)
    no_punct_tokens = new_tokens
    
    return no_punct_tokens

In [ ]:
def remove_stopwords(tweet, stopword_file):
    with open(stopword_file, 'r', encoding='utf-8') as f:
        stop_words = f.read().splitlines()

    # Tokenize the tweet
    tokens = tweet.split()

    no_stopword_tokens = []
    for token in tokens:
        if token not in stop_words:
            no_stopword_tokens.append(token)
    
    # Join the tokens back into a processed tweet
    processed_tweet = " ".join(no_stopword_tokens)
    
    return processed_tweet

In [ ]:
def clean_tweet2(tweet):
    # Remove mentions (@username)
    tweet = re.sub(r"@[A-Za-z0-9_]+", "", tweet)
    # Remove retweets (RT)
    tweet = re.sub(r"RT\s+", "", tweet)
    # Remove URLs (http or https)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", "", tweet)
    # Remove any remaining non-Arabic characters
    tweet = re.sub(r"[^؀-ۿ]+", " ", tweet)
    # Remove extra whitespace
    tweet = re.sub(r"\s+", " ", tweet.strip())
    return tweet

In [ ]:
def clean_tweet(tweet):
    tweet = normalize_hamza(tweet)
    tweet = strip_tatweel(tweet)
    tweet = strip_tashkeel(tweet)


    tweet = re.sub("[ًٌٍَُِّْٰ]", "", tweet)
    tweet = re.sub("[إأٱآا]", "ا", tweet)
    tweet = re.sub("ى", "ي", tweet)
    tweet = re.sub("ؤ", "ء", tweet)
    tweet = re.sub("ئ", "ء", tweet)
    tweet = re.sub("ة", "ه", tweet)
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', tweet)
    tweet = tweet.lower()
    return tweet

In [ ]:
def normalize_hamza(text):
    """Normalize Alef with Hamza Above and Alef with Hamza Below to Alef"""
    text = re.sub("[أإآا]", "ا", text)
    return text

In [ ]:
import csv

# Open the input and output CSV files
with open('ArSarcasm_train.csv', 'r', encoding='utf-8') as f_input, \
        open('output_train.csv', 'w', newline='', encoding='utf-8') as f_output:
    # Define the CSV reader and writer objects
    csv_reader = csv.reader(f_input)
    csv_writer = csv.writer(f_output)
    
    # Read the header row and add a new 'cleaned_tweet' column
    header = next(csv_reader)
    header.append('cleaned_tweet')
    csv_writer.writerow(header)
    
    # Loop over each row in the input CSV file
    for row in csv_reader:
        # Get the tweet text from the row
         tweet = row[4]
        # Clean the tweet text using the cleaning functions
         no_stopword_tokens = remove_stopwords(tweet, '/content/list.txt')
         tweet = clean_tweet(no_stopword_tokens)
         tweet = clean_tweet2(tweet)
         tweet = remove_emojis(tweet)
         tokens = tokenize_tweet(tweet)
         no_punct_tokens = remove_punctuation(tokens)
         cleaned_tweet = ' '.join(no_punct_tokens)
        
        # Add the cleaned tweet text to the row
         row.append(cleaned_tweet)
        
        # Write the updated row to the output CSV file
         csv_writer.writerow(row)

In [ ]:
df = pd.read_csv("/content/output_train.csv")
df.head()

,dialect,sarcasm,sentiment,original_sentiment,tweet,source,cleaned_tweet
0,gulf,False,negative,negative,"""نصيحه ما عمرك اتنزل لعبة سوبر ماريو مش زي ما ...",semeval,نصيحه عمرك اتنزل لعبه سوبر ماريو مش زي كنا متو...
1,msa,False,neutral,positive,"""#نادين_نسيب_نجيم ❤️❤️❤️مجلة #ماري_كلير 💭#ملكة...",semeval,نادين نسيب نجيم مجله ماري كلير ملكه الصحراء
2,egypt,False,neutral,neutral,"""@Alito_NBA اتوقع انه بيستمر""",semeval,اتوقع بيستمر
3,levant,True,neutral,negative,"""@KSA24 يعني ""بموافقتنا"" لأن دمشق صايرة موسكو""",semeval,يعني بموافقتنا دمشق صايره موسكو
4,msa,False,neutral,negative,"""RT @alaahmad20: قائد في الحرس يعترف بفقدان ال...",semeval,قاءد الحرس يعترف بفقدان السيطره الامنيه شرقي و...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8437 entries, 0 to 8436
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   dialect             8437 non-null   object
 1   sarcasm             8437 non-null   bool  
 2   sentiment           8437 non-null   object
 3   original_sentiment  8437 non-null   object
 4   tweet               8437 non-null   object
 5   source              8437 non-null   object
 6   cleaned_tweet       8429 non-null   object
dtypes: bool(1), object(6)
memory usage: 403.8+ KB


In [ ]:
df = df.dropna(axis=0,subset= ['dialect','sarcasm','sentiment','original_sentiment','tweet','source','cleaned_tweet'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8429 entries, 0 to 8436
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   dialect             8429 non-null   object
 1   sarcasm             8429 non-null   bool  
 2   sentiment           8429 non-null   object
 3   original_sentiment  8429 non-null   object
 4   tweet               8429 non-null   object
 5   source              8429 non-null   object
 6   cleaned_tweet       8429 non-null   object
dtypes: bool(1), object(6)
memory usage: 469.2+ KB


In [ ]:
df.to_csv('output_train_modified.csv', index=False)  # Save the modified DataFrame to a new CSV file

In [ ]:
df = pd.read_csv("/content/output_train_modified.csv")
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8429 entries, 0 to 8428
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   dialect             8429 non-null   object
 1   sarcasm             8429 non-null   bool  
 2   sentiment           8429 non-null   object
 3   original_sentiment  8429 non-null   object
 4   tweet               8429 non-null   object
 5   source              8429 non-null   object
 6   cleaned_tweet       8429 non-null   object
dtypes: bool(1), object(6)
memory usage: 403.5+ KB


In [ ]:
import csv

def read_clean_tweets(filename):
    """
    Read a CSV file and extract a column called "clean_tweet" into a list.
    
    Args:
    filename - string representing the name of the CSV file
    
    Returns:
    clean_tweets - list of strings representing clean tweets
    """
    clean_tweets = []

    with open(filename, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            clean_tweet = row['cleaned_tweet']
            clean_tweets.append(clean_tweet)
            
    return clean_tweets

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

def extract_features(data, word_embedding_model):
    """
    Extract TF-IDF features from preprocessed text data using word embeddings.
    
    Args:
    data - list of preprocessed text data
    word_embedding_model - pre-trained word embedding model
    
    Returns:
    feature_matrix - sparse matrix of TF-IDF features
    """
    # Get the vocabulary from the word embedding model
    vocabulary = set(word_embedding_model.wv.key_to_index.keys())
   

    # Initialize TfidfVectorizer object with vocabulary
    tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=5, ngram_range=(1, 2), vocabulary=vocabulary)

    # Fit and transform data to obtain feature matrix
    feature_matrix = tfidf_vectorizer.fit_transform(data)

    # Save the tfidf_vectorizer for future use
    joblib.dump(tfidf_vectorizer, '/content/drive/MyDrive/tfidf_vectorizer_sarcasm2.sav')

    return feature_matrix

In [ ]:
import csv

def read_sentiment_labels(filename):
    """
    Read a CSV file and extract a column called "sentiment" into a list.
    
    Args:
    filename - string representing the name of the CSV file
    
    Returns:
    sentiment_labels - list of strings representing sentiment labels
    """
    sentiment_labels = []

    with open(filename, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            sentiment_label = row['sentiment']
            sentiment_labels.append(sentiment_label)

    return sentiment_labels

In [ ]:
import csv

def read_sarcasm_labels(filename):

    sarcasm_labels = []

    with open(filename, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            sarcasm_label = row['sarcasm']
            sarcasm_labels.append(sarcasm_label)

    return sarcasm_labels

In [ ]:
tfidf_vectorizer = joblib.load('/content/drive/MyDrive/tfidf_vectorizer_sarcasm2.sav')
xx = read_clean_tweets("/content/output_train_modified.csv")
X = tfidf_vectorizer.transform(df['cleaned_tweet'])
y = read_sentiment_labels("/content/output_train_modified.csv")
z = read_sarcasm_labels("/content/output_train_modified.csv")
print(len(z))

8429


# **SVM (Sarcasm Detection & Sentiment Analysis)**

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import joblib

In [ ]:
# Combine sentiment and sarcasm labels into a single label
combined_labels = [sentiment + '_' + sarcasm for sentiment, sarcasm in zip(y, z)]

# Train an SVM model for sentiment and sarcasm analysis
svm = SVC(kernel='linear')
svm.fit(X, combined_labels)

# Evaluate the performance of the SVM model
y_pred = svm.predict(X)
accuracy = accuracy_score(combined_labels, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(combined_labels, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

# Save the trained model
joblib.dump(svm, '/content/drive/MyDrive/svm_sentiment_sarcasm_model2.sav')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.9004626883378811
Precision: 0.9021367745873715
Recall: 0.9004626883378811
F1-score: 0.8927160911999452


['/content/drive/MyDrive/svm_sentiment_sarcasm_model2.sav']

In [ ]:
import csv



# Open the input and output CSV files
with open('ArSarcasm_test.csv', 'r', encoding='utf-8') as f_input, \
        open('output_test.csv', 'w', newline='', encoding='utf-8') as f_output:
    # Define the CSV reader and writer objects
    csv_reader = csv.reader(f_input)
    csv_writer = csv.writer(f_output)
    
    # Read the header row and add a new 'cleaned_tweet' column
    header = next(csv_reader)
    header.append('cleaned_tweet')
    csv_writer.writerow(header)
    
    # Loop over each row in the input CSV file
    for row in csv_reader:
        # Get the tweet text from the row
         tweet = row[4]
        # Clean the tweet text using the cleaning functions
         no_stopword_tokens = remove_stopwords(tweet, '/content/list.txt')
         tweet = clean_tweet(no_stopword_tokens)
         tweet = clean_tweet2(tweet)
         tweet = remove_emojis(tweet)
         tokens = tokenize_tweet(tweet)
         no_punct_tokens = remove_punctuation(tokens)
         cleaned_tweet = ' '.join(no_punct_tokens)
        
        # Add the cleaned tweet text to the row
         row.append(cleaned_tweet)
        
        # Write the updated row to the output CSV file
         csv_writer.writerow(row)

In [ ]:
df = pd.read_csv("/content/output_test.csv")
df.head()

,dialect,sarcasm,sentiment,original_sentiment,tweet,source,cleaned_tweet
0,msa,True,negative,negative,"""@AbuEmad74241481 @Cesars2014 هههههه حزب الله ...",semeval,هههههه حزب الله حطموا اسطوره الميركافا الاسراء...
1,gulf,False,positive,neutral,"""RT @JannetForster: البنات اللي م صامو بقولكم ...",semeval,البنات اللي صامو بقولكم ترا رمضان قرب قليل الل...
2,msa,True,neutral,neutral,اشارة رابعة اشبه بنار تحرق الانقلابيين,astd,اشاره رابعه اشبه بنار تحرق الانقلابيين
3,msa,False,neutral,neutral,"""@EGYPTAIR ماهي مميزات درجه بزنس علماً اني في ...",semeval,ماهي مميزات درجه بزنس علما اني بروكسيل اريد ال...
4,msa,False,neutral,neutral,ما لا تراه على التلفاز منافسة شديدة بين المرشح...,semeval,تراه التلفاز منافسه شديده المرشحين هيلاري كلنت...


In [ ]:
tfidf_vectorizer = joblib.load('/content/drive/MyDrive/tfidf_vectorizer_sarcasm2.sav')

In [ ]:
xx = read_clean_tweets("/content/output_test.csv")
tweet_features = tfidf_vectorizer.transform(df['cleaned_tweet'])
y = read_sentiment_labels("/content/output_test.csv")
z = read_sarcasm_labels("/content/output_test.csv")

In [ ]:
svm = joblib.load('/content/drive/MyDrive/svm_sentiment_sarcasm_model2.sav')
y_true_combined = [sentiment + '_' + sarcasm for sentiment, sarcasm in zip(y, z)]

# Make predictions on the test data
y_pred_combined = svm.predict(tweet_features)

# Calculate accuracy, precision, recall, and F1-score
accuracy = accuracy_score(y_true_combined, y_pred_combined)
precision, recall, f1, _ = precision_recall_fscore_support(y_true_combined, y_pred_combined, average='weighted')
report = classification_report(y_true_combined, y_pred_combined)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Classification Report:")
print(report)

Accuracy: 0.5729857819905213
Precision: 0.5578532396391767
Recall: 0.5729857819905213
F1-score: 0.5288367065149547
Classification Report:
                precision    recall  f1-score   support

negative_False       0.45      0.31      0.37       417
 negative_True       0.48      0.25      0.33       299
 neutral_False       0.60      0.88      0.71      1045
  neutral_True       1.00      0.03      0.06        33
positive_False       0.60      0.28      0.39       303
 positive_True       0.00      0.00      0.00        13

      accuracy                           0.57      2110
     macro avg       0.52      0.29      0.31      2110
  weighted avg       0.56      0.57      0.53      2110



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

# **Decison Trees (Sarcasm Detection & Sentiment Analysis)**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# Combine sentiment and sarcasm labels into a single label
combined_labels = [sentiment + '_' + sarcasm for sentiment, sarcasm in zip(y, z)]

# Define the decision tree classifier
dt_classifier = DecisionTreeClassifier()

# Define the parameter grid for grid search
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
}

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(dt_classifier, param_grid, cv=5)
grid_search.fit(X, combined_labels)

# Get the best classifier with the optimal hyperparameters
dt_best_classifier = grid_search.best_estimator_

# Train the decision tree model with the best hyperparameters
dt_best_classifier.fit(X, combined_labels)

# Evaluate the performance of the decision tree model on the training set
y_pred = dt_best_classifier.predict(X)
accuracy = accuracy_score(combined_labels, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(combined_labels, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

# Save the trained model
joblib.dump(dt_best_classifier, '/content/drive/MyDrive/dt_sentiment_sarcasm_model.sav')

Accuracy: 0.5960374896191719
Precision: 0.6164768203632366
Recall: 0.5960374896191719
F1-score: 0.5427648732843509


['/content/drive/MyDrive/dt_sentiment_sarcasm_model.sav']

In [ ]:
dt= joblib.load('/content/drive/MyDrive/dt_sentiment_sarcasm_model.sav')
y_true_combined = [sentiment + '_' + sarcasm for sentiment, sarcasm in zip(y, z)]

# Make predictions on the test data
y_pred_combined = dt.predict(tweet_features)

# Calculate accuracy, precision, recall, and F1-score
accuracy = accuracy_score(y_true_combined, y_pred_combined)
precision, recall, f1, _ = precision_recall_fscore_support(y_true_combined, y_pred_combined, average='weighted')
report = classification_report(y_true_combined, y_pred_combined)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Classification Report:")
print(report)

Accuracy: 0.5184834123222749
Precision: 0.45723463296121
Recall: 0.5184834123222749
F1-score: 0.4488373552128162
Classification Report:
                precision    recall  f1-score   support

negative_False       0.34      0.15      0.20       417
 negative_True       0.37      0.22      0.27       299
 neutral_False       0.56      0.89      0.69      1045
  neutral_True       0.00      0.00      0.00        33
positive_False       0.42      0.14      0.21       303
 positive_True       0.00      0.00      0.00        13

      accuracy                           0.52      2110
     macro avg       0.28      0.23      0.23      2110
  weighted avg       0.46      0.52      0.45      2110



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

# **Naive Bayes (Sarcasm Detection & Sentiment Analysis)**

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

# Combine sentiment and sarcasm labels into a single label
combined_labels = [sentiment + '_' + sarcasm for sentiment, sarcasm in zip(y, z)]

# Define the Naive Bayes classifier
nb_classifier = MultinomialNB()

# Define the parameter grid for grid search
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0],  # Smoothing parameter
    'fit_prior': [True, False]  # Whether to learn class prior probabilities
}

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(nb_classifier, param_grid, cv=5)
grid_search.fit(X, combined_labels)

# Get the best classifier with the optimal hyperparameters
nb_best_classifier = grid_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:")
print(grid_search.best_params_)

# Train the Naive Bayes model with the best hyperparameters
nb_best_classifier.fit(X, combined_labels)

# Evaluate the performance of the Naive Bayes model on the training set
y_pred = nb_best_classifier.predict(X)
accuracy = accuracy_score(combined_labels, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(combined_labels, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

# Save the trained model
joblib.dump(nb_best_classifier, '/content/drive/MyDrive/nb_sentiment_sarcasm_model.sav')


Best Hyperparameters:
{'alpha': 0.1, 'fit_prior': False}
Accuracy: 0.9133942341914818
Precision: 0.9209953477957414
Recall: 0.9133942341914818
F1-score: 0.9099460661177752


['/content/drive/MyDrive/nb_sentiment_sarcasm_model.sav']

In [ ]:
nb= joblib.load('/content/drive/MyDrive/nb_sentiment_sarcasm_model.sav')
y_true_combined = [sentiment + '_' + sarcasm for sentiment, sarcasm in zip(y, z)]

# Make predictions on the test data
y_pred_combined = nb.predict(tweet_features)

# Calculate accuracy, precision, recall, and F1-score
accuracy = accuracy_score(y_true_combined, y_pred_combined)
precision, recall, f1, _ = precision_recall_fscore_support(y_true_combined, y_pred_combined, average='weighted')
report = classification_report(y_true_combined, y_pred_combined)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Classification Report:")
print(report)

Accuracy: 0.5630331753554503
Precision: 0.5429919151656407
Recall: 0.5630331753554503
F1-score: 0.5076966158105143
Classification Report:
                precision    recall  f1-score   support

negative_False       0.48      0.26      0.33       417
 negative_True       0.49      0.23      0.31       299
 neutral_False       0.58      0.91      0.71      1045
  neutral_True       0.50      0.03      0.06        33
positive_False       0.57      0.22      0.32       303
 positive_True       0.00      0.00      0.00        13

      accuracy                           0.56      2110
     macro avg       0.44      0.27      0.29      2110
  weighted avg       0.54      0.56      0.51      2110



# **Testing Each Model Individually**

In [ ]:
tweet = "السحب الليلة على الايفون .. رتويت للمرفقة وطبق الشروط 👇"
no_stopword_tokens = remove_stopwords(tweet, '/content/list.txt')
print(no_stopword_tokens)
tweet = clean_tweet(no_stopword_tokens)
tweet = clean_tweet2(tweet)
tweet = remove_emojis(tweet)
tokens = tokenize_tweet(tweet)
no_punct_tokens = remove_punctuation(tokens)
cleaned_tweet = ' '.join(no_punct_tokens)
print(cleaned_tweet)

السحب الليلة الايفون .. رتويت للمرفقة وطبق الشروط 👇
السحب الليله الايفون رتويت للمرفقه وطبق الشروط


In [ ]:
tweet_features = tfidf_vectorizer.transform([cleaned_tweet])


prediction = svm.predict(tweet_features)
label_parts = prediction[0].split('_')
if label_parts[1] == 'False':
    output = 'not-sarcastic'
else:
    output = 'sarcastic'

print(label_parts[0])
print(output)

neutral
not-sarcastic


In [ ]:
tweet_features = tfidf_vectorizer.transform([cleaned_tweet])


prediction = dt.predict(tweet_features)
label_parts = prediction[0].split('_')
if label_parts[1] == 'False':
    output = 'not-sarcastic'
else:
    output = 'sarcastic'

print(label_parts[0])
print(output)

neutral
not-sarcastic


In [ ]:
tweet_features = tfidf_vectorizer.transform([cleaned_tweet])


prediction = nb.predict(tweet_features)
label_parts = prediction[0].split('_')
if label_parts[1] == 'False':
    output = 'not-sarcastic'
else:
    output = 'sarcastic'

print(label_parts[0])
print(output)

neutral
not-sarcastic


# **SVM (Sarcasm Detection)**

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import joblib

# Train the SVM model on sarcasm data only
svm = SVC(kernel='linear')
svm.fit(X, z)

# Evaluate the performance of the SVM model on sarcasm data
y_pred = svm.predict(X)
accuracy = accuracy_score(z, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(z, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

# Save the trained model
joblib.dump(svm, '/content/drive/MyDrive/svm_sarcasm_only_model.sav')



Accuracy: 0.9422232767825365
Precision: 0.9452439939701524
Recall: 0.9422232767825365
F1-score: 0.9369639241946666


['/content/drive/MyDrive/svm_sarcasm_only_model.sav']

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import joblib

# Load the trained SVM model
svm = joblib.load('/content/drive/MyDrive/svm_sarcasm_only_model.sav')

# Make predictions on the test data
y_pred_sarcasm = svm.predict(tweet_features)

# Calculate accuracy, precision, recall, and F1-score
accuracy = accuracy_score(z, y_pred_sarcasm)
precision, recall, f1, _ = precision_recall_fscore_support(z, y_pred_sarcasm, average='weighted')
report = classification_report(z, y_pred_sarcasm)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Classification Report:")
print(report)


Accuracy: 0.8417061611374408
Precision: 0.8063789867498634
Recall: 0.8417061611374408
F1-score: 0.7940792586072297
Classification Report:
              precision    recall  f1-score   support

       False       0.85      0.98      0.91      1765
        True       0.58      0.11      0.19       345

    accuracy                           0.84      2110
   macro avg       0.72      0.55      0.55      2110
weighted avg       0.81      0.84      0.79      2110



# **Naive Bayes(Sarcasm Detection)**

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import joblib

# Define the Naive Bayes classifier
nb_classifier = MultinomialNB()

# Define the parameter grid for grid search
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 2.0],  # Smoothing parameter
    'fit_prior': [True, False]  # Whether to learn class prior probabilities
}

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(nb_classifier, param_grid, cv=5)
grid_search.fit(X, z)

# Get the best classifier with the optimal hyperparameters
nb_best_classifier = grid_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:")
print(grid_search.best_params_)

# Train the Naive Bayes model with the best hyperparameters
nb_best_classifier.fit(X, z)

# Evaluate the performance of the Naive Bayes model on the training set
y_pred = nb_best_classifier.predict(X)
accuracy = accuracy_score(z, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(z, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

# Save the trained model
joblib.dump(nb_best_classifier, '/content/drive/MyDrive/nb_sarcasm_only_model.sav')


Best Hyperparameters:
{'alpha': 0.1, 'fit_prior': False}
Accuracy: 0.93830822161585
Precision: 0.9423256834274332
Recall: 0.93830822161585
F1-score: 0.9320014147959964


['/content/drive/MyDrive/nb_sarcasm_only_model.sav']

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import joblib

# Load the trained Naive Bayes model
nb = joblib.load('/content/drive/MyDrive/nb_sarcasm_only_model.sav')

# Make predictions on the test data
y_pred_sarcasm = nb.predict(tweet_features)

# Calculate accuracy, precision, recall, and F1-score
accuracy = accuracy_score(z, y_pred_sarcasm)
precision, recall, f1, _ = precision_recall_fscore_support(z, y_pred_sarcasm, average='weighted')
report = classification_report(z, y_pred_sarcasm)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Classification Report:")
print(report)


Accuracy: 0.8369668246445497
Precision: 0.7909268199157837
Recall: 0.8369668246445497
F1-score: 0.7847158537981224
Classification Report:
              precision    recall  f1-score   support

       False       0.85      0.98      0.91      1765
        True       0.51      0.08      0.14       345

    accuracy                           0.84      2110
   macro avg       0.68      0.53      0.53      2110
weighted avg       0.79      0.84      0.78      2110

